# FedEM Attack Analysis

TJ Kim
11.5.21

#### Summary:
- Load the locally trained model and FedEM model
- nodes = 3, mixtures = 3

In [1]:
cd /home/ubuntu/FedEM/

/home/ubuntu/FedEM


### Import Relevant Libraries
Take it from the run_experiment.py folder

In [2]:
# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from torch.utils.tensorboard import SummaryWriter
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *

from transfer_attacks.TA_utils import *
from transfer_attacks.Boundary_Transferer import *

### Generate Aggregator Pre-requisite
- Clients, Test Clients, Ensemble_Learner
- Follow through the code in run_experiment.py

In [29]:
# Manually set argument parameters
args_ = Args()
args_.experiment = "cifar10"
args_.method = "FedEM"
args_.decentralized = False
args_.sampling_rate = 1.0
args_.input_dimension = None
args_.output_dimension = None
args_.n_learners= 3
args_.n_rounds = 10
args_.bz = 128
args_.local_steps = 1
args_.lr_lambda = 0
args_.lr =0.03
args_.lr_scheduler = 'multi_step'
args_.log_freq = 10
args_.device = 'cpu'
args_.optimizer = 'sgd'
args_.mu = 0
args_.communication_probability = 0.1
args_.q = 1
args_.locally_tune_clients = False
args_.seed = 1234
args_.verbose = 1
args_.save_path = 'weights/cifar/21_12_30_feddef2_n40_linf0_5/'
args_.validation = False

# Generate the dummy values here
aggregator, clients = dummy_aggregator(args_, num_user=40)

==> Clients initialization..
===> Building data iterators..


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 185.74it/s]


===> Initializing clients..


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:33<00:00,  2.39it/s]


==> Test Clients initialization..
===> Building data iterators..


0it [00:00, ?it/s]


===> Initializing clients..


0it [00:00, ?it/s]


++++++++++++++++++++++++++++++
Global..
Train Loss: 2.292 | Train Acc: 12.195% |Test Loss: 2.292 | Test Acc: 12.291% |
++++++++++++++++++++++++++++++++++++++++++++++++++
################################################################################


### Generate Data 

In [5]:
def load_client_data(clients, c_id, mode = 'test'):
    
    data_x = []
    data_y = []

    if mode == 'all': # load all validation sets together
        for i in range(len(clients)):
            daniloader = clients[i].val_iterator
            for (x,y,idx) in daniloader.dataset:
                data_x.append(x)
                data_y.append(y)
    else:
        if mode == 'train':
            daniloader = clients[c_id].train_iterator
        if mode == 'val':
            daniloader = clients[c_id].val_iterator
        else:
            daniloader = clients[c_id].test_iterator

        for (x,y,idx) in daniloader.dataset:
            data_x.append(x)
            data_y.append(y)

    data_x = torch.stack(data_x)
    data_y = torch.stack(data_y)        

    dataloader = Custom_Dataloader(data_x, data_y)
    
    return dataloader

In [40]:
clients[0].learners_ensemble.learners[0].model

MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

In [38]:
dataloader = load_client_data(clients, 2, mode = "val")
dataloader.y_data.shape[0]

776

In [4]:
# Combine Validation Data across all clients as test
data_x = []
data_y = []

for i in range(len(clients)):
    daniloader = clients[i].val_iterator
    for (x,y,idx) in daniloader.dataset:
        data_x.append(x)
        data_y.append(y)

data_x = torch.stack(data_x)
data_y = torch.stack(data_y)

In [5]:
# Create dataloader from validation dataset that allows for diverse batch size
dataloader = Custom_Dataloader(data_x, data_y)

### Load Model of specific type

In [6]:
# Change name if need be
args_.save_path = 'weights/cifar/21_12_30_feddef_n40_linf0_5/'

# Import weights for aggregator
aggregator.load_state(args_.save_path)

# This is where the models are stored -- one for each mixture --> learner.model for nn
hypotheses = aggregator.global_learners_ensemble.learners

# obtain the state dict for each of the weights 
weights_h = []

for h in hypotheses:
    weights_h += [h.model.state_dict()]

In [7]:
weight_name = args_.save_path + 'train_client_weights.npy'
weights = np.load(weight_name)
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

#print(weights)

# Set model weights
model_weights = []
num_models = 40

for i in range(num_models):
    model_weights += [weights[i]]
    
    
# Generate the weights to test on as linear combinations of the model_weights
models_test = []

for (w0,w1,w2) in model_weights:
    # first make the model with empty weights
    new_model = copy.deepcopy(hypotheses[0].model)
    new_model.eval()
    new_weight_dict = copy.deepcopy(weights_h[0])
    for key in weights_h[0]:
        new_weight_dict[key] = w0*weights_h[0][key] + w1*weights_h[1][key] + w2*weights_h[2][key]
    new_model.load_state_dict(new_weight_dict)
    models_test += [new_model]

### Set Up Transfer Attack Scenario
- Set up order of which attacks will take place -- keep the same transferer, but simply swap out the weights of the adversary and flush out the existing data points analysis was done on
- Make dictionaries beforehand recording all of the metrics

In [8]:
# Set Up Dictionaries -- list holds the adversary idx
logs_adv = []

for i in range(len(model_weights)):
    adv_dict = {}
    adv_dict['orig_acc_transfers'] = None
    adv_dict['orig_similarities'] = None
    adv_dict['adv_acc_transfers'] = None
    adv_dict['adv_similarities'] = None
    adv_dict['orig_target_hit'] = None
    adv_dict['adv_target_hit'] = None
    adv_dict['metric_variance'] = None
    adv_dict['metric_alignment'] = None
    adv_dict['metric_ingrad'] = None
    logs_adv += [adv_dict]

In [9]:
# Make transferer and Assign model index
victim_idxs =[0,1,2,3,4,5,6]# [0,41,42,43,44,45,46] #  Include other models for analysis?
custom_batch_size = 500

for adv_idx in victim_idxs:
    print("id", adv_idx)
    # Perform Attack
    
    # Import client dataloader here (aka. adv_idx)
    dataloader = load_client_data(clients, c_id, mode = 'val') # or test/train
    t1 = Transferer(models_list=models_test, dataloader=dataloader)
    
    batch_size = min(custom_batch_size, dataloader.y_data.shape[0])
    
    # atk Params setup
    t1.atk_params.set_params(batch_size=batch_size, eps=0.1, alpha=0.05, iteration = 30,
                   target = 5, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x))
    
    t1.atk_params = PGD_Params()
    t1.atk_params.set_params(batch_size=batch_size, iteration = 30,
                   target = 5, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x),
                   step_size = 0.05, step_norm = "inf", eps = 0.4, eps_norm = "inf")

    
    t1.generate_victims(victim_idxs)
    t1.generate_advNN(adv_idx)
    t1.generate_xadv(atk_type = "pgd")
    t1.send_to_victims(victim_idxs)
    # t1.check_empirical_metrics(orig_flag = True)
    # t1.check_empirical_metrics()
    
    # Log Performance
    logs_adv[adv_idx]['orig_acc_transfers'] = t1.orig_acc_transfers
    logs_adv[adv_idx]['orig_similarities'] = t1.orig_similarities
    logs_adv[adv_idx]['adv_acc_transfers'] = t1.adv_acc_transfers
    logs_adv[adv_idx]['adv_similarities'] = t1.adv_similarities
    logs_adv[adv_idx]['orig_target_hit'] = t1.orig_target_hit
    logs_adv[adv_idx]['adv_target_hit'] = t1.adv_target_hit
    
    logs_adv[adv_idx]['metric_variance'] = t1.metric_variance
    logs_adv[adv_idx]['metric_alignment'] = t1.metric_alignment
    logs_adv[adv_idx]['metric_ingrad'] = t1.metric_ingrad

id 0
id 1
id 2
id 3
id 4
id 5
id 6


### Print and Record Results
- Organize the results into a matrix and print them into an excel sheet
- Following Categories (sim_benign, sim_adv, target_adv,grad_alignment)

In [10]:
metrics = ['orig_acc_transfers','orig_similarities','adv_similarities','adv_target_hit','metric_alignment']

orig_acc = np.zeros([len(victim_idxs),len(victim_idxs)]) 
sim_benign = np.zeros([len(victim_idxs),len(victim_idxs)]) 
sim_adv = np.zeros([len(victim_idxs),len(victim_idxs)]) 
target_adv = np.zeros([len(victim_idxs),len(victim_idxs)]) 
# grad_align = np.zeros([len(victim_idxs),len(victim_idxs)]) 

In [11]:
for adv_idx in range(len(victim_idxs)):
    for victim in range(len(victim_idxs)):
        orig_acc[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[0]][victim_idxs[victim]].data.tolist()
        sim_benign[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[1]][victim_idxs[victim]].data.tolist()
        sim_adv[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[2]][victim_idxs[victim]].data.tolist()
        target_adv[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[3]][victim_idxs[victim]].data.tolist()
#         grad_align[adv_idx,victim] = logs_adv[adv_idx][metrics[4]][victim].data.tolist()

In [12]:
# Save to Excel file

## convert your array into a dataframe
orig_acc_df = pd.DataFrame(orig_acc)
sim_benign_df = pd.DataFrame(sim_benign)
sim_adv_df = pd.DataFrame(sim_adv)
target_adv_df = pd.DataFrame(target_adv)
# grad_align_df = pd.DataFrame(grad_align)

## save to xlsx file

#filepath = 'my_excel_file.xlsx'

# df.to_excel(filepath, index=False)

In [13]:
orig_acc_df

,0,1,2,3,4,5,6
0,0.662,0.798,0.718,0.670,0.638,0.738,0.750
1,0.672,0.750,0.728,0.668,0.672,0.738,0.740
2,0.726,0.798,0.750,0.692,0.664,0.756,0.764
3,0.668,0.768,0.722,0.682,0.632,0.738,0.738
4,0.698,0.780,0.740,0.680,0.640,0.754,0.764
5,0.690,0.730,0.718,0.676,0.646,0.726,0.730
6,0.700,0.770,0.734,0.674,0.648,0.752,0.752


In [14]:
sim_benign_df

,0,1,2,3,4,5,6
0,1.000,0.756,0.858,0.556,0.510,0.820,0.810
1,0.776,1.000,0.886,0.636,0.566,0.906,0.912
2,0.856,0.882,1.000,0.612,0.538,0.976,0.964
3,0.574,0.664,0.626,1.000,0.746,0.622,0.634
4,0.496,0.550,0.514,0.734,1.000,0.514,0.514
5,0.862,0.894,0.970,0.586,0.504,1.000,0.984
6,0.852,0.904,0.962,0.626,0.540,0.990,1.000


In [15]:
sim_adv_df

,0,1,2,3,4,5,6
0,1.000,0.816,0.962,0.446,0.338,0.952,0.944
1,0.538,1.000,0.726,0.202,0.104,0.796,0.834
2,0.900,0.922,1.000,0.398,0.170,0.974,0.976
3,0.706,0.696,0.746,1.000,0.400,0.732,0.724
4,0.790,0.502,0.714,0.558,1.000,0.676,0.654
5,0.838,0.940,0.980,0.332,0.164,1.000,0.996
6,0.780,0.944,0.970,0.312,0.156,0.992,1.000


In [16]:
target_adv_df

,0,1,2,3,4,5,6
0,0.879195,0.709172,0.854586,0.304251,0.205817,0.843400,0.834452
1,0.417978,0.876405,0.593258,0.096629,0.026966,0.662921,0.703371
2,0.820346,0.841991,0.909091,0.311688,0.088745,0.913420,0.911255
3,0.616071,0.629464,0.676339,0.747768,0.220982,0.669643,0.662946
4,0.703057,0.368996,0.600437,0.353712,0.762009,0.561135,0.539301
5,0.716854,0.820225,0.844944,0.200000,0.056180,0.858427,0.858427
6,0.677778,0.817778,0.844444,0.202222,0.057778,0.862222,0.866667


In [17]:
# grad_align_df

In [18]:
# Average all the information together and present
orig_acc_mean = np.mean(orig_acc)
sim_benign_mean = np.mean(sim_benign)
sim_adv_mean = np.mean(sim_adv_df)
target_adv_mean = np.mean(target_adv)
# grad_align_mean = np.mean((grad_align.sum(1)-0)/(grad_align.shape[1]-0))

print("orig_acc_mean", orig_acc_mean)
print("Sim Benign Mean", sim_benign_mean)
# print("Sim ADV Mean", sim_adv_mean)
print("Target ADV Mean", target_adv_mean)
# print("Grad Align Mean", grad_align_mean)

orig_acc_mean 0.7151428923314932
Sim Benign Mean 0.7690204400189069
Target ADV Mean 0.6040968042703307


In [19]:
# Calculate average distance norm across examples
delta = t1.x_adv - t1.x_orig
dnorm = delta.view(delta.shape[0], -1).norm(float(2), dim=1)
print(torch.mean(dnorm))
print(torch.max(dnorm))

tensor(14.8801, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(17.0992, device='cuda:0', grad_fn=<MaxBackward1>)


In [20]:
# IFSGM
4.5347 # mean
4.8902 # Max
0.8203 # Mean transfer rate

0.8203

In [21]:
# PGD
4.4978 # mean perturbation
4.5000 # Max perturbation
0.8594 # Mean transfer rate

0.8594

In [22]:
# Print image of L2 norm of different values